# Descriptor

### 학습 내용
- descriptor가 무엇인지 이해하고 어떻게 효율적으로 동작하는지 이해한다.
- data descriptor vs non-data descriptor 의 개념적 차이와 세부구현의 차이를 분석
- descriptor를 활용한 코드 재사용 방법
- 좋은 활용 예

학습에 앞서 먼저 descriptor의 syntax를 알아보고자 한다.
https://docs.python.org/3/howto/descriptor.html<br>
위 사이트에서 공부하였음

가장 쉬운 예제는 아래와 같다<br>
A descripotr that returns a constant

In [1]:
class Ten:
    def __get__(self, obj, objtype=None):
        return 10

To use the desciptor, it must be stored as a class Variable in another class<br>

In [2]:
class A:
    x = 5
    y = Ten()

In [3]:
a = A()
print(a.x)
print(a.y)

5
10


In the a.x attribute lookup, the dot operator finds the key x<br>
and the value 5 in the class dictionary.<br>

In the a.y lookup, the dot operator finds a descripotr instance, <br>
recognized by its \_\_get__ method, and calls that method which returns 10<br>

Note that the value 10 is not stored in either the class or the instance dic<br>
Instead, the value 10 in computed on demand

and we're gonna see dynamic lookup

Interesting descriptor typically run computation instead of returning constant<br>

In [4]:
import os

class DirectorySize:
    def __get__(self, obj, objtype=None):
        return len(os.listdir(obj.dirname))

class Directory:
    size = DirectorySize()

    def __init__(self, dirname):
        self.dirname = dirname

In [5]:
s = Directory('../')
j = Directory('../joono')

print(s.size)
print(j.size)

9
8


위와 같은 예제를 살펴보면 \_\_get__ method가 어떻게 동작하는지, 그 목적이 무엇인지 알 수 있다.<br>

descriptor가 가장 자주 사용되는 부분은 instance의 data에 접근을 제어하는 일이다.<br>
Descriptor는 public attribute에 저장되고 실제 data는 다른 instance dict에 저장된다.<br>
그리고 descriptor의 \_\_get__, \_\_set__ methods는 public attribute에 접근할 때 호출된다.<br>

아래의 예제에서 _age_는 public attribute,<br>
_\_age_는 private attribute이다.<br>
그리고 age에 access할 때 마다 descriptor가 lookup, update를 log한다.

In [6]:
import logging

logging.basicConfig(level=logging.INFO)

class LoggedAgeAccess:

    def __get__(self, obj, objtype=None):
        value = obj._age
        logging.info('Accessing %r giving %r', 'age', value)
        return value

    def __set__(self, obj, value):
        logging.info('Updating %r to %r', 'age', value)
        obj._age = value

class Person:

    age = LoggedAgeAccess()             # Descriptor instance

    def __init__(self, name, age):
        self.name = name                # Regular instance attribute
        self.age = age                  # Calls __set__()

    def birthday(self):
        self.age += 1

아래 결과를 보면 regular attribute인 name에 access할 때는 아무런 log도 남지 않지만<br>
Descriptor로 정의된 age에 접근할 때면 desciptor가 log를 남긴다.

_obj_ parameter는 g나 s같은 instance of Dictionary이고 이는 target directory가 무엇인지<br>
\_\_get__ method 에게 알려주는 역할을 한다.<br>
_objtype_ parameter는 class Dictionary이다.<br>

In [7]:
mary = Person('Mary M', 30)
david = Person('David D', 40)

INFO:root:Updating 'age' to 30
INFO:root:Updating 'age' to 40


In [8]:
print(vars(mary))
print(vars(david))

{'name': 'Mary M', '_age': 30}
{'name': 'David D', '_age': 40}


In [9]:
mary.age

INFO:root:Accessing 'age' giving 30


30

In [10]:
david.age

INFO:root:Accessing 'age' giving 40


40

그러나 위와 같은 방법의 문제점은 딱 age 하나 밖에 descriptor를 설정하지 못한다는 단점이 있다.
아래에서 위와 같은 문제를 해결해보자

descriptor를 사용할 때 각 descriptor에게 어떤 변수명이 사용됐는지 알려줄 수 있다.<br>
_Person_ class는 2개의 name, age라는 descriptor instance를 가지고 있다.<br>

_Person_ class가 정의될 때, \_\_set_name__이라는 함수를 callback한다.<br>
name, age에게 이들의 public, private 변수명을 알려주기 위해서이다.

In [56]:
import logging

logging.basicConfig(level=logging.INFO)

class LoggedAccess:
    def __set_name__(self, owner, name):
        self.public_name = name
        self.private_name = 'my_' + name
        self.private_name2 = '--' + name

    def __get__(self, instance, owner):
        value = getattr(instance, self.private_name)
        print(value)
        logging.info(f'Accessing {self.public_name} to {value}')
        return value

    def __set__(self, instance, value):
        logging.info(f'Updating {self.public_name} to {value}')
        setattr(instance, self.private_name, value)

class Person:
    name = LoggedAccess()
    age = LoggedAccess()

    def __init__(self, name, age):
        self.name = name
        self.age = age

    def birthday(self):
        self.age += 1

Person classr가 정의될 때, 각 public, private name들을 저장하기 위해 \_\_set_name__이 callback 되었다.<br>
vars 함수를 통해 정말 그렇게 되었는지 확인해보자.<br>

descriptor는 'dot operator'에 의해서만 invoke 된다.<br>
따라서 아래의 두 코드에서는 log가 발생되지 않는다.

In [57]:
joono = Person('joono', 26)
rion = Person('rion', 35)
vars(joono)

INFO:root:Updating name to joono
INFO:root:Updating age to 26
INFO:root:Updating name to rion
INFO:root:Updating age to 35


{'my_name': 'joono', 'my_age': 26}

In [58]:
vars(rion)

{'my_name': 'rion', 'my_age': 35}

그러나 아래의 코드에서는 모두 dot operator가 사용되므로 log가 발생했다.

In [62]:
pete = Person('Pete P', 10)
kate = Person('Kate K', 20)

print(pete.__dict__)

INFO:root:Updating name to Pete P
INFO:root:Updating age to 10
INFO:root:Updating name to Kate K
INFO:root:Updating age to 20


{'_name': 'Pete P', '_age': 10}


descriptor instance에는 private name만 contain하게 된다.

In [58]:
print(vars(pete))
print(vars(kate))

{'_name': 'Pete P', '_age': 10}
{'_name': 'Kate K', '_age': 20}


## Closing thought
- descriptor는 \_\_get__, \_\_set__, \_\_delete__ method를 정의하는 class를 지칭하며,<br>
optional 하게 \_\_set_name__ 도 포함할 수 있다.
- descriptor는 dot operator에 의해서만 invoke 된다.
- 오직 class variable로 선언될 때만 사용 가능하다. instance로 선언되면 아무런 효과도 없다.
- descriptor의 motivation은 attrubute를 lookup할 때 발생하는 상황에 대하여 통제할 수 있도록<br>
 class variable에 objects를 저장할 수 있는 hook을 제공하는 것이다.<br>
- classmethod, staticmethod, property등등이 descriptor로서 사용된다.

- 아래는 본문에서 제시한 유효성을 검사하는 객체를 어떻게 만드는지 보여주는 예이다.
- 유효성 검사 함수는 자유롭게 생성할 수 있으며, 객체에 값을 할당하기 전에 실행된다.

In [16]:
class Validation:
    def __init__(self, validation_func, error_msg:str):
        self.validation_func = validation_func
        self.error_msg = error_msg

    def __call__(self, value):
        if not self.validation_func(value):
            raise ValueError(f'{value!r} {self.error_msg}')

class Field:    # descriptor
    def __init__(self, *validations):
        self._name = None
        self.validations = validations

    def __set_name__(self, owner, name):
        self._name = name

    def __get__(self, instance, owner):
        if instance is None:    # classㄴ에서 호출
            return self
        return instance.__dict__[self._name]

    def validate(self, value):
        for validation in self.validations:
            validation(value)

    def __set__(self, instance, value):
        self.validate(value)
        instance.__dict__[self._name] = value

class ClientClass:
    descriptor = Field(
        Validation(lambda x: isinstance(x, (int, float)), '는 숫자가 아님'),
        Validation(lambda x: x >= 0, "는 0보다 작음")
    )

In [17]:
client = ClientClass()
client.descriptor = 42
client.descriptor

42

In [18]:
client.descriptor = -42

ValueError: -42 는 0보다 작음

In [ ]:
client.descriptor = 'invalid value'

여기서 기억할 점은 property 자리에 놓일 수 있는 것은 descriptor로 추상화할 수 있으며<br>
여러번 재사용이 가능하다는 것이다.

다음 예제는 이 메서드를 사용하여 관리자 권한이 없는 객체에서 속성을 제거하지 못하도록 하는 디스크립터를 만든다.<br>

In [ ]:
class ProtectedAttribute:
    def __init__(self, requires_role=None) -> None:
        self.permission_required = requires_role
        self._name = None

    def __set_name__(self, owner, name):
        self._name = name

    def __set__(self, user, value):
        if value is None:
            raise ValueError(f'{self._name}를 None으로 설정할 수 없음')
        user.__dict__[self._name] = value

    def __delete__(self, user):
        if self.permission_required in user.permissions:
            user.__dict__[self._name] = None
        else:
            raise ValueError(f'{user!r} 사용자는 {self.permission_required} 권한이 없음')

class User:
    """admin 권한을 가진 사용자만 이메일을 주소를 삭제할 수 있음."""
    email = ProtectedAttribute(requires_role="admin")

    def __init__(self, user_name: str, email: str, permissions: list=None) -> None:
        self.username = user_name
        self.email = email
        self.permissions = permissions or []

    def __str__(self):
        return self.username

In [ ]:
admin = User('root', 'root@d.com', ['admin'])
user = User('user', 'user1@d.com', ['email', 'helpdesk'])
admin.email

In [ ]:
del admin.email
admin.email is None

In [ ]:
user.email

In [ ]:
del user.email

### \_\_set_name__
- 일반적으로 하위 호환을 위해 \_\_init__에 기본 값을 지정하고 \_\_set_name__ 를 함께 사용하는 것이 좋다

# descriptor의 유형

- data descrptor : \_\_set__이나 \_\_delete__ method를 구현함
- non-data descriptor : \_\_get__만 구현함
- \_\_set_name__ 은 위 분류에 영향을 미치지 않는다.

### non-data descriptor

In [ ]:
class NonDataDescriptor:
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return 42

class ClientClass:
    descriptor = NonDataDescriptor()

In [ ]:
client = ClientClass()
client.descriptor

In [ ]:
client.descriptor = 43
client.descriptor

여기서 descriptor를 지우고 다시 lookup하게 되면 어떤 결과를 얻게 될까?

In [ ]:
del client.descriptor
client.descriptor

In [ ]:
vars(client)

.descriptor attribute을 lookup하면 client.\_\_dict__에서 'descriptor'를 찾지 못하고<br>
마침내 클래스에서 디스크립터를 찾게 된다. 이것이 \_\_get__ method가 호출되는 이유이다.

그러나 \_\_set__ method가 정의되지 않으면 인스턴스의 사전이 변경되므로 client.\_\_dict__ 가 변경된다.<br>

In [ ]:
client.descriptor = 99
vars(client)

### data descriptor
data descriptor와의 차이를 살펴보기 위해 새로운 클래스를 정의하자

In [ ]:
import logging
logger = logging.getLogger()

class DataDescriptor:
    def __get__(self, instance, owner):
        if instance is None:    # class에서 호출
            return self
        return 42

    def __set__(self, instance, value):
        logger.debug(f'{instance!r}.descriptor를 {value!r} 값으로 설정')
        instance.__dict__['descriptor'] = value

class ClientClass:
    descriptor = DataDescriptor()

In [ ]:
client = ClientClass()
client.descriptor

In [ ]:
client.descriptor = 99
client.descriptor

결과가 변하지 않았다. 이렇게 되는 이유는 set을 호출할 때 instance의 dictionary에 값을 저장하기 때문이다.<br>
그리고 data descriptor에서 attribute을 lookup할 때는 dict에서 조회하는게 아니라<br>
class의 descriptor를 먼저 조회한다.

In [ ]:
del client.descriptor

\_\_delete__ method가 정의되지 않았으므로 del 도 실행되지 않는다.

# descriptor 실전

In [ ]:
class Traveller:
    def __init__(self, name, current_city):
        self.name = name
        self._current_city = current_city
        self._cities_visited = [current_city]

    @property
    def current_city(self):
        return self._current_city
    @current_city.setter
    def current_city(self, new_city):
        if new_city != self._current_city:
            self._cities_visited.append(new_city)
            self._current_city = new_city

    @property
    def cities_visited(self):
        return self._cities_visited

In [ ]:
alice = Traveller('Alice', 'Barcelona')
alice.current_city = 'Paris'
alice.current_city = 'Brussels'
alice.current_city = 'Amsterdam'
alice.cities_visited

만약 alice가 사용하였던 티켓, 가계부를 작성한다면 이와 같이 또다른 property를 선언하여<br>
값을 계속 update 해야 할 것이다.

- attribute에 대해 이름을 가진 일반적인 descriptor를 만들 것이다.

In [67]:
class HistoryTracedAttribute:
    def __init__(self, trace_attribute_name) -> None:
        self.trace_attribute_name = trace_attribute_name
        self._name = None

    def __set_name__(self, owner, name):
        self._name = name

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return instance.__dict__[self._name]

    def __set__(self, instance, value):
        self._track_change_in_value_for_instance(instance, value)
        instance.__dict__[self._name] = value

    def _track_change_in_value_for_instance(self, instance, value):
        self._set_default(instance)
        if self._need_to_track_change(instance, value):
            instance.__dict__[self.trace_attribute_name].append(value)

    def _need_to_track_change(self, instance, value) -> bool:
        try:
            current_value = instance.__dict__[self._name]
        except KeyError:
            return True
        return value != current_value

    def _set_default(self, instance):
        instance.__dict__.setdefault(self.trace_attribute_name, [])

class Traveller:
    current_city = HistoryTracedAttribute('cities_visited')

    def __init__(self, name, current_city):
        self.name = name
        self.current_city = current_city

In [70]:
joono = Traveller('joono', 'Seoul')
joono.current_city = 'Goyang'
vars(joono)

{'name': 'joono',
 'cities_visited': ['Seoul', 'Goyang'],
 'current_city': 'Goyang'}

In [26]:
class Student:
    def __init__(self, name, grade):
        self.name = name
        self.grade = grade

In [28]:
s = Student('joono', 3)
vars(s)

{'name': 'joono', 'grade': 3}

```python
def object_getattribute(obj, name):
    "Emulate PyObject_GenericGetAttr() in Objects/object.c"
    null = object()
    objtype = type(obj)
    cls_var = getattr(objtype, name, null)
    descr_get = getattr(type(cls_var), '__get__', null)
    if descr_get is not null:
        if (hasattr(type(cls_var), '__set__')
            or hasattr(type(cls_var), '__delete__')):
            return descr_get(cls_var, obj, objtype)     # data descriptor
    if hasattr(obj, '__dict__') and name in vars(obj):
        return vars(obj)[name]                          # instance variable
    if descr_get is not null:
        return descr_get(cls_var, obj, objtype)         # non-data descriptor
    if cls_var is not null:
        return cls_var                                  # class variable
    raise AttributeError(name)
```

위 코드는 attribute에 access할 때 거치는 pure python equivalent이다.

우선 순위는 아래와 같다.
1. data descriptor<br>
2. instance variable<br>
3. non-data descriptor<br>
4. class variable<br>
이래도 없다면 AttrubuteError를 raise한다.

### Automatic name notification
새로운 클래스가 생성될 때 type이라는 metaclass가 해당 클래스의 \_\_dict__를 scan한다.<br>
이때 하나의 entry라도 descriptor가 존재하고 여기에 \_\_set_name__ method가 존재한다면<br>
_owner_m _name_ 의 arguments로 해당 method를 호출한다.<br>
_owner_는 해당 새로 생성된 클래스이며, name은 descriptor로 할당된 variable이다.<br>

### ORM example

In [19]:
import sqlite3
conn = sqlite3.connect('entertainment.db')

class Field:

    def __set_name__(self, owner, name):
        self.fetch = f'SELECT {name} FROM {owner.table} WHERE {owner.key}=?;'
        self.store = f'UPDATE {owner.table} SET {name}=? WHERE {owner.key}=?;'

        print(self.fetch)
        print(self.store)

    def __get__(self, obj, objtype=None):
        return conn.execute(self.fetch, [obj.key]).fetchone()[0]

    def __set__(self, obj, value):
        conn.execute(self.store, [value, obj.key])
        conn.commit()

In [20]:
class Movie:
    table = 'Movies'                    # Table name
    key = 'title'                       # Primary key
    director = Field()
    year = Field()

    def __init__(self, key):
        self.key = key

class Song:
    table = 'Music'
    key = 'title'
    artist = Field()
    year = Field()
    genre = Field()

    def __init__(self, key):
        self.key = key

SELECT director FROM Movies WHERE title=?;
UPDATE Movies SET director=? WHERE title=?;
SELECT year FROM Movies WHERE title=?;
UPDATE Movies SET year=? WHERE title=?;
SELECT artist FROM Music WHERE title=?;
UPDATE Music SET artist=? WHERE title=?;
SELECT year FROM Music WHERE title=?;
UPDATE Music SET year=? WHERE title=?;
SELECT genre FROM Music WHERE title=?;
UPDATE Music SET genre=? WHERE title=?;


In [24]:
jaws = Movie('jaws')

In [25]:
jaws.__dict__

{'key': 'jaws'}

In [26]:
vars(jaws)

{'key': 'jaws'}

In [27]:
jaws.table

'Movies'